## Day 1: Trebuchet?!

In [53]:
sample = """1abc2
pqr3stu8vwx
a1b2c3d4e5f
treb7uchet""".split('\n')
sample

['1abc2', 'pqr3stu8vwx', 'a1b2c3d4e5f', 'treb7uchet']

In [55]:
import re

def extract_calibration_value(txt, debug=False):
    digits = re.match(r'^.*?(\d)', txt).group(1), re.match(r'.*(\d).*$', txt).group(1)
    if debug:
        print(digits)
    return int(''.join(digits))

sample_res = [12, 38, 15, 77]
assert all(extract_calibration_value(line, debug=True) == res for line, res in zip(sample, sample_res))

('1', '2')
('3', '8')
('1', '5')
('7', '7')


In [41]:
with open('input_day1.txt') as f:
    lines = f.readlines()
len(lines), lines[-1]

(1000, 'mbnfjkxptbtjmgcrtkhxjvjhjnine83mpnsixfcmxcbnspx\n')

In [42]:
sum(map(extract_calibration_value, lines))

53974

In [61]:
sample = """two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen""".split('\n')
sample_res = [29, 83, 13, 24, 42, 14, 76]

DIGIT_STRINGS = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
DIGIT_MAP = {s: str(i+1) for i, s in enumerate(DIGIT_STRINGS)}
DIGIT_REGEX = r'(\d|' + '|'.join(DIGIT_STRINGS) + ')'
print(DIGIT_REGEX)

def extract_calibration_value_2(txt, debug=False):
    digits = re.match(rf'^.*?{DIGIT_REGEX}', txt).group(1), re.match(rf'.*{DIGIT_REGEX}.*$', txt).group(1)
    digits = [DIGIT_MAP.get(d) or d for d in digits]
    if debug:
        print(digits)
    return int(''.join(digits))

assert all(extract_calibration_value_2(line, debug=True) == res for line, res in zip(sample, sample_res))

(\d|one|two|three|four|five|six|seven|eight|nine)
['2', '9']
['8', '3']
['1', '3']
['2', '4']
['4', '2']
['1', '4']
['7', '6']


In [62]:
sum(map(extract_calibration_value_2, lines))

52840